In [50]:
import pickle5 as pickle
import numpy as np

T = 200
p = 1
n_iterations = 200
prior = "svss"
sparsity = 0.40

In [121]:
with open(f'../simulations/results/statistics_{T}_{p}_{n_iterations}_{prior}_{sparsity}.pkl', 'rb') as f:
        simulation_result = pickle.load(f)

In [122]:
# Select one of the results from the TVP for a comparison

# tvp_bar_result = np.array([1.409606, 1.439558, 1.443909, 1.460863, 1.459694, 1.461891, 1.460617, 1.463583]) # 100 0.20 M = 3
# tvp_bar_result = np.array([1.425317, 1.451723, 1.453873, 1.461871, 1.463032, 1.469662, 1.463004, 1.465282]) # 100 0.40 M = 3
# tvp_bar_result = np.array([1.744158, 1.751534, 1.762016, 1.755724, 1.759277, 1.776115, 1.780216, 1.790951]) # 200 0.20 M = 3
# tvp_bar_result = np.array([1.763981, 1.769919, 1.783354, 1.783171, 1.788644, 1.803967, 1.804127, 1.821872]) # 200 0.40 M = 3

# tvp_bar_result = np.array([2.203113, 2.266640, 2.307729, 2.323546, 2.341028, 2.362956, 2.385565, 2.401525]) # 100 0.20 M = 7
# tvp_bar_result = np.array([2.240987, 2.319214, 2.354877, 2.361660, 2.384902, 2.406189, 2.429912, 2.450251]) # 100 0.40 M = 7
# tvp_bar_result = np.array([3.041940, 3.091507, 3.119626, 3.138423, 3.151080, 3.172323, 3.196318, 3.218827]) # 200 0.20 M = 7
# tvp_bar_result = np.array([3.185229, 3.270465, 3.310410, 3.340728, 3.399988, 3.452003, 3.507754, 3.621743]) # 200 0.40 M = 7

In [123]:
simulation_result_block = np.block(simulation_result[0])

In [124]:
M_3 = simulation_result_block[:T]
M_7 = simulation_result_block[T:]

In [125]:
result_block = M_3

threshold_percentage = 10e-2

threshold_high = int(round(n_iterations - threshold_percentage*n_iterations,0))
threshold_low = 0 #int(round(threshold_percentage*n_iterations,0))

msfe_list = result_block[:,:8].astype(np.float64).mean(1)

cleaned_indices = msfe_list.argsort()[threshold_low:threshold_high]
msfe_list_cleaned = msfe_list[cleaned_indices]

msfe_h_step = result_block[cleaned_indices,:8].astype(np.float64).mean(0)

msd = result_block[cleaned_indices,16].astype(np.float64).mean()

In [126]:
msd = round(msd,6)
msfe = round(np.median(msfe_list_cleaned),3)

print(f"MSFE = {msfe} & MSD = {msd}")

MSFE = 2.04 & MSD = 0.04694


In [127]:
msfe_ratio = np.round(msfe_h_step/tvp_bar_result,3)

print(f"VI/TVP = {msfe_ratio}")

VI/TVP = [1.22  1.257 1.251 1.249 1.245 1.225 1.222 1.209]
